In [102]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
import gc
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [103]:
import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [104]:
# Consider the training dataset
# Split the data up in n sets, based on `search_id` to prevent occurence of a `search_id` in multiple sets
#   CF score-matrices can be computed here as well
# Use crossvalidation on the n sets to select hyperparams
# Finally train model on full training dataset and make a prediction of the (unseen) test dataset

In [141]:
import util.data

In [171]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=1*1000)
data_test = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)
# scores = pd.read_csv('data/scores_train.csv', sep=';')

In [172]:
data = data.drop(columns=['position'])

In [173]:
ids = sklearn.utils.shuffle(data.srch_id.unique(), random_state=123)
N = ids.size
N # total number of samples

40

In [174]:
y_labels = [ k for k in data.columns if k not in data_test.columns ]
y_labels

['click_bool', 'booking_bool', 'score']

In [145]:
# y_data = data[['click_bool', 'booking_bool', 'score']].copy()
# x_data = data.drop(columns=y_labels)
# TODO split xy at the end

In [170]:
# # clear memory
# data = None
# gc.collect();

In [147]:
n = 5 # number of cv splits
selection_size = np.floor(N/n).astype(int)
a = selection_size
# note that the final splice may be smaller
ids_selections = [ ids[i*a: min((i+1)*a, N)] for i in range(n) ]
assert len(ids_selections) == n
print(selection_size)

8


In [177]:
data_splits = [ data.loc[data.srch_id.isin(srch_ids)] for srch_ids in ids_selections ]
# x_data_splits = [ x_data.loc[x_data.srch_id.isin(srch_ids)] for srch_ids in ids_selections ]
# y_data_splits = [ y_data.iloc[x.index] for x in x_data_splits ]
# y_data_splits = [ y_data.loc[y_data.srch_id.isin(srch_ids)] for srch_ids in ids_selections ]
len(data_splits)

5

In [149]:
# we can now use (n-1) of the splits to train and the one other split to validate
# Now we will sample from a split to prevent class imbalance

In [156]:
# split slices again, into classes: select 1/3 booking, 1/3 click (but no booking), 1/3 none

In [178]:
# x_train = x_data_splits[0]
# y_train = y_data_splits[0]
# assert x_train.shape[0] == y_train.shape[0]

In [240]:
def split_bookings_clicks_others(data):
    bookings = data.query('booking_bool == 1')
    clicks = data.query('click_bool == 1 and booking_bool != 0')
    others = data.query('click_bool != 1')
    return bookings, clicks, others
    
def sample(datasets=[], size_per_sample=100):
    samples = [ data.loc[np.random.choice(data.index, size_per_sample)] 
                for data in datasets
              ]
    order_dataset_samples = pd.concat(samples)
    return order_dataset_samples.sample(frac=1)
    
def split_xy(data: pd.DataFrame):
    return data.drop(columns=y_labels), data[y_labels]

In [241]:
bco_splits = [ split_bookings_clicks_others(data) for data in data_splits ]

# for every cv split i
i = 0
bco_split = bco_splits[i]
bookings, clicks, others = bco_split
n_max = max([xy.shape[0] for xy in bco_split ])
n_min = min([xy.shape[0] for xy in bco_split ])
n_max, n_min

(203, 5)

In [242]:
sample([bookings, clicks, others], size_per_sample=2)

,srch_id,visitor_hist_starrating,prop_id,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,promotion_flag,srch_length_of_stay,...,srch_booking_window_bin1,srch_booking_window_bin2,orig_destination_distance_bin0,orig_destination_distance_bin1,orig_destination_distance_bin2,visitor_hist_adr_usd_bin0,visitor_hist_adr_usd_bin1,price_usd_bin0,price_usd_bin1,price_usd_bin2
223,25,0.0,66209,0.0,1,0.217778,1.650691,-0.860465,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
543,46,0.0,90440,0.5,1,-0.208889,0.004608,1.476744,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
243,28,0.0,13996,0.0,1,0.053333,-0.183410,0.186047,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
243,28,0.0,13996,0.0,1,0.053333,-0.183410,0.186047,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
935,74,0.0,12870,0.0,1,-0.613333,0.000000,0.465116,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
211,25,0.0,23228,0.0,0,0.217778,2.752074,-0.546512,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [249]:
# cv_folds = []
# for bco in bco_splits:
#     n_max = max([df.shape[0] for df in bco_split ])
#     cv_folds.append(split_xy(sample(bco, n_max)))


# x_train, y_train = split_xy(sample)
model = sklearn.ensemble.RandomForestClassifier(n_estimators=100)

cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='accuracy') # roc_auc accuracy
# sampler()

,click_bool,booking_bool,score
211,1,1,5
243,1,1,5
243,1,1,5
225,0,0,0
211,1,1,5
243,1,1,5
211,1,1,5
251,0,0,0
243,1,1,5
944,1,1,5


In [209]:
# sample splitted splices
index = sklearn.utils.resample(y_bookings.index, replace=True, random_state=123)
y_bookings.loc[index]
# note the duplicate entries

,click_bool,booking_bool,score
543,1,1,5
944,1,1,5
543,1,1,5
243,1,1,5
899,1,1,5


In [169]:
# TODO sample n_max items during training epochs

sklearn.utils.resample(y_bookings, y_clicks, y_others, replace=True, random_state=123)

ValueError: Found input variables with inconsistent numbers of samples: [5, 5, 203]

In [ ]:
model = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
scores_acc = cross_val_score(model, x_train, y_train, cv=n_folds, scoring='accuracy') # roc_auc accuracy